In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Capa Bronce - Uber NYC Data Pipeline
# MAGIC 
# MAGIC ## Objetivo
# MAGIC Ingesta de datos crudos desde archivos CSV y almacenamiento en formato Delta Lake con metadatos de auditoría.
# MAGIC 
# MAGIC ## Responsabilidades
# MAGIC - Leer todos los archivos CSV de Uber NYC (abril-septiembre 2014)
# MAGIC - Preservar los datos en su formato original
# MAGIC - Añadir metadatos de auditoría (timestamp, archivo fuente)
# MAGIC - Guardar en formato Delta Lake
# MAGIC 
# MAGIC ## Entrada
# MAGIC - Archivos CSV en: `/Volumes/workspace/default/uber_etl_azure/`
# MAGIC 
# MAGIC ## Salida
# MAGIC - Tabla Delta: `uber_bronze` en `/Volumes/workspace/default/uber_etl_azure/bronze/`



In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Configuración e Imports

# COMMAND ----------

In [0]:
# Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

In [0]:
# Configurar rutas
source_path = "/Volumes/workspace/default/uber_etl_azure/"
bronze_path = "/Volumes/workspace/default/uber_etl_azure/bronze/"

In [0]:
# Crear directorio si no existe
dbutils.fs.mkdirs(bronze_path)

True

In [0]:
print("✅ Configuración de Capa Bronce inicializada")
print(f"📁 Datos fuente: {source_path}")
print(f"🥉 Destino Bronce: {bronze_path}")

✅ Configuración de Capa Bronce inicializada
📁 Datos fuente: /Volumes/workspace/default/uber_etl_azure/
🥉 Destino Bronce: /Volumes/workspace/default/uber_etl_azure/bronze/


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Validación de Archivos Fuente

# COMMAND ----------

In [0]:
# Listar archivos en el directorio fuente
try:
    files = dbutils.fs.ls(source_path)
    csv_files = [f for f in files if f.name.endswith('.csv')]
    
    print("📋 Archivos CSV encontrados:")
    for file in csv_files:
        print(f"   📄 {file.name} - {file.size} bytes")
    
    print(f"\n📊 Total archivos CSV: {len(csv_files)}")
    
    if len(csv_files) == 0:
        raise Exception("❌ No se encontraron archivos CSV en el directorio fuente")
        
except Exception as e:
    print(f"❌ Error al acceder al directorio fuente: {str(e)}")
    raise

📋 Archivos CSV encontrados:
   📄 uber-raw-data-apr14.csv - 26110064 bytes
   📄 uber-raw-data-aug14.csv - 38324585 bytes
   📄 uber-raw-data-jul14.csv - 36870544 bytes
   📄 uber-raw-data-jun14.csv - 30695600 bytes
   📄 uber-raw-data-may14.csv - 30174084 bytes
   📄 uber-raw-data-sep14.csv - 47540750 bytes

📊 Total archivos CSV: 6


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Definición de Esquema

# COMMAND ----------

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Definir esquema esperado para los archivos CSV
expected_schema = StructType([
    StructField("Date/Time", StringType(), True),
    StructField("Lat", DoubleType(), True),
    StructField("Lon", DoubleType(), True),
    StructField("Base", StringType(), True)
])

print("📋 Esquema esperado definido:")
print(expected_schema)

📋 Esquema esperado definido:
StructType([StructField('Date/Time', StringType(), True), StructField('Lat', DoubleType(), True), StructField('Lon', DoubleType(), True), StructField('Base', StringType(), True)])


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. Ingesta de Datos

# COMMAND ----------

In [0]:
# Leer todos los archivos CSV del directorio fuente
print("📥 Iniciando ingesta de datos crudos...")

try:
    # Leer archivos CSV con metadatos
    df_bronze = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("pathGlobFilter", "*.csv") \
        .load(source_path)
    
    # Verificar que se leyeron datos
    if df_bronze.count() == 0:
        raise Exception("❌ No se encontraron datos en los archivos CSV")
    
    print("✅ Archivos CSV leídos exitosamente")
    
except Exception as e:
    print(f"❌ Error durante la lectura de archivos: {str(e)}")
    raise

📥 Iniciando ingesta de datos crudos...
✅ Archivos CSV leídos exitosamente


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. Añadir Metadatos de Auditoría

# COMMAND ----------

In [0]:
# Añadir metadatos de auditoría
print("🏷️ Añadiendo metadatos de auditoría...")

df_bronze = df_bronze \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("source_file", regexp_extract(col("_metadata.file_path"), "([^/]+\\.csv)$", 1)) \
    .withColumn("bronze_layer_version", lit("1.0"))

print("✅ Metadatos de auditoría añadidos")

🏷️ Añadiendo metadatos de auditoría...
✅ Metadatos de auditoría añadidos


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. Validación de Datos

# COMMAND ----------

In [0]:
# Mostrar información sobre los datos ingestados
print("📊 ESTADÍSTICAS DE INGESTA:")
print("=" * 40)

record_count = df_bronze.count()
print(f"📊 Total de registros ingestados: {record_count:,}")

print(f"\n📋 Esquema de datos:")
df_bronze.printSchema()

print(f"\n📋 Muestra de datos (primeras 5 filas):")
df_bronze.select("Date/Time", "Lat", "Lon", "Base", "source_file", "ingestion_timestamp").show(5, truncate=False)

📊 ESTADÍSTICAS DE INGESTA:
📊 Total de registros ingestados: 4,534,327

📋 Esquema de datos:
root
 |-- Date/Time: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)
 |-- source_file: string (nullable = false)
 |-- bronze_layer_version: string (nullable = false)


📋 Muestra de datos (primeras 5 filas):
+----------------+-------+--------+------+-----------------------+-------------------------+
|Date/Time       |Lat    |Lon     |Base  |source_file            |ingestion_timestamp      |
+----------------+-------+--------+------+-----------------------+-------------------------+
|8/1/2014 0:03:00|40.7366|-73.9906|B02512|uber-raw-data-aug14.csv|2025-07-28 17:06:20.63562|
|8/1/2014 0:09:00|40.726 |-73.9918|B02512|uber-raw-data-aug14.csv|2025-07-28 17:06:20.63562|
|8/1/2014 0:12:00|40.7209|-74.0507|B02512|uber-raw-data-aug14.csv|2025-07-28 17:06:20.63562|
|8/1/2

In [0]:
# Validar distribución por archivo fuente
print(f"\n📊 Distribución de registros por archivo:")
df_bronze.groupBy("source_file").count().orderBy("source_file").show(10, truncate=False)


📊 Distribución de registros por archivo:
+-----------------------+-------+
|source_file            |count  |
+-----------------------+-------+
|uber-raw-data-apr14.csv|564516 |
|uber-raw-data-aug14.csv|829275 |
|uber-raw-data-jul14.csv|796121 |
|uber-raw-data-jun14.csv|663844 |
|uber-raw-data-may14.csv|652435 |
|uber-raw-data-sep14.csv|1028136|
+-----------------------+-------+



In [0]:
# Detectar valores nulos
print(f"\n🔍 Análisis de valores nulos:")
null_counts = []
for col_name in ["Date/Time", "Lat", "Lon", "Base"]:
    null_count = df_bronze.filter(col(col_name).isNull()).count()
    null_counts.append((col_name, null_count))
    print(f"   {col_name}: {null_count:,} nulos")

print("✅ Validación de datos completada")


🔍 Análisis de valores nulos:
   Date/Time: 0 nulos
   Lat: 0 nulos
   Lon: 0 nulos
   Base: 0 nulos
✅ Validación de datos completada


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 7. Guardar en Capa Bronce

# COMMAND ----------

In [0]:
# Guardar en formato Delta en la capa Bronce
print("💾 Guardando datos en la capa Bronce...")

try:
    df_bronze.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save(f"{bronze_path}uber_bronze")
    
    print("✅ Datos guardados exitosamente en formato Delta")
except Exception as e:
    print(f"❌ Error al guardar los datos: {e}")

💾 Guardando datos en la capa Bronce...
✅ Datos guardados exitosamente en formato Delta


In [0]:
# Verificar que los datos se guardaron correctamente
try:
    verification_df = spark.read.format("delta").load(f"{bronze_path}uber_bronze")
    verification_count = verification_df.count()
    
    print(f"🔍 Verificación: {verification_count:,} registros en la tabla Delta")
    
    if verification_count != record_count:
        raise Exception(f"❌ Error de integridad: se esperaban {record_count:,} registros, pero se encontraron {verification_count:,}")
    
    print("✅ Verificación de integridad exitosa")
    
except Exception as e:
    print(f"❌ Error al guardar en capa Bronce: {str(e)}")
    raise

🔍 Verificación: 4,534,327 registros en la tabla Delta
✅ Verificación de integridad exitosa


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Resumen y Estadísticas Finales

# COMMAND ----------

In [0]:
print("📊 RESUMEN FINAL - CAPA BRONCE")
print("=" * 50)
print(f"📁 Archivos procesados: {len(csv_files)}")
print(f"📊 Registros totales: {record_count:,}")
print(f"💾 Formato de salida: Delta Lake")
print(f"📍 Ubicación: {bronze_path}uber_bronze")
print(f"🕐 Procesamiento completado: {current_timestamp()}")

📊 RESUMEN FINAL - CAPA BRONCE
📁 Archivos procesados: 6
📊 Registros totales: 4,534,327
💾 Formato de salida: Delta Lake
📍 Ubicación: /Volumes/workspace/default/uber_etl_azure/bronze/uber_bronze
🕐 Procesamiento completado: Column<'current_timestamp()'>


In [0]:
# Mostrar estadísticas por columna
print(f"\n📈 ESTADÍSTICAS POR COLUMNA:")
df_bronze.describe("Lat", "Lon").show()

print("\n✅ CAPA BRONCE COMPLETADA EXITOSAMENTE")
print("🎉 Los datos están listos para ser procesados por la Capa Plata")


📈 ESTADÍSTICAS POR COLUMNA:
+-------+--------------------+------------------+
|summary|                 Lat|               Lon|
+-------+--------------------+------------------+
|  count|             4534327|           4534327|
|   mean|   40.73926075399743|-73.97301934563407|
| stddev|0.039949910576595304|0.0572666973547159|
|    min|             39.6569|           -74.929|
|    max|             42.1166|          -72.0666|
+-------+--------------------+------------------+


✅ CAPA BRONCE COMPLETADA EXITOSAMENTE
🎉 Los datos están listos para ser procesados por la Capa Plata


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 9. Registro de Tabla (Opcional)

# COMMAND ----------

In [0]:
# Registrar la tabla como vista temporal para consultas
df_bronze.createOrReplaceTempView("uber_bronze_temp")

print("📋 Tabla registrada como vista temporal: 'uber_bronze_temp'")
print("💡 Ahora puedes usar SQL para consultar los datos:")
print("   SELECT * FROM uber_bronze_temp LIMIT 10;")

📋 Tabla registrada como vista temporal: 'uber_bronze_temp'
💡 Ahora puedes usar SQL para consultar los datos:
   SELECT * FROM uber_bronze_temp LIMIT 10;


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 10. Comandos Útiles para Monitoreo

# COMMAND ----------

In [0]:
# Comandos útiles para monitorear la tabla Delta
print("🔧 COMANDOS DE MONITOREO:")
print("=" * 30)

🔧 COMANDOS DE MONITOREO:


In [0]:
# Mostrar historial de la tabla Delta (si existe)
try:
    history_df = spark.sql(f"DESCRIBE HISTORY delta.`{bronze_path}uber_bronze`")
    print("📜 Historial de la tabla Delta:")
    history_df.select("version", "timestamp", "operation", "operationMetrics").show(5, truncate=False)
except:
    print("ℹ️ Historial no disponible (primera ejecución)")

📜 Historial de la tabla Delta:
+-------+-------------------+---------+------------------------------------------------------------------------------------------------------------------------+
|version|timestamp          |operation|operationMetrics                                                                                                        |
+-------+-------------------+---------+------------------------------------------------------------------------------------------------------------------------+
|3      |2025-07-28 17:06:36|WRITE    |{numFiles -> 4, numRemovedFiles -> 4, numRemovedBytes -> 22062114, numOutputRows -> 4534327, numOutputBytes -> 22062114}|
|2      |2025-07-26 20:04:31|WRITE    |{numFiles -> 4, numRemovedFiles -> 4, numRemovedBytes -> 22062110, numOutputRows -> 4534327, numOutputBytes -> 22062114}|
|1      |2025-07-22 03:41:04|WRITE    |{numFiles -> 4, numRemovedFiles -> 4, numRemovedBytes -> 22060334, numOutputRows -> 4534327, numOutputBytes -> 22062110}|
|0 

In [0]:
# Mostrar detalles de la tabla
print(f"\n📋 Detalles de la tabla:")
try:
    detail_df = spark.sql(f"DESCRIBE DETAIL delta.`{bronze_path}uber_bronze`")
    detail_df.select("format", "numFiles", "sizeInBytes", "location").show(1, truncate=False)
except:
    print("ℹ️ Detalles no disponibles")

print("\n✅ Monitoreo completado")
print("🚀 Listo para ejecutar el notebook de Capa Plata")


📋 Detalles de la tabla:
+------+--------+-----------+-----------------------------------------------------------------+
|format|numFiles|sizeInBytes|location                                                         |
+------+--------+-----------+-----------------------------------------------------------------+
|delta |4       |22062114   |dbfs:/Volumes/workspace/default/uber_etl_azure/bronze/uber_bronze|
+------+--------+-----------+-----------------------------------------------------------------+


✅ Monitoreo completado
🚀 Listo para ejecutar el notebook de Capa Plata
